In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh, minres, LinearOperator
from scipy.sparse import csr_matrix
import time
from utils.tools import build_weighted_bethe_hessian
from scipy.sparse.csgraph import minimum_spanning_tree

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
adjacency_matrix.data = adjacency_matrix.data ** 0.3
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()

In [3]:
r = np.sqrt(np.mean(degrees**2) / np.mean(degrees) - 1)
#r = np.mean(adjacency_matrix.data**2)
D, A = build_weighted_bethe_hessian(adjacency_matrix, r)

n = adjacency_matrix.shape[0]
I = scipy.sparse.eye(n, format='csr')
Hr = D - A #+ I * np.mean(degrees)

In [4]:
class MatrixOperator(object):

    def __init__(self, A):
        self.A = A.astype(PETSc.ScalarType)
        self.n_calls = 0

    def mult(self, A, x, y):
        xx = x.getArray(readonly=1)
        yy = y.getArray(readonly=0)
        yy[:] = self.A.dot(xx)
        self.n_calls += 1
        
    def getDiagonal(self, A, y):
        yy = y.getArray(readonly=0)
        yy[:] = self.A.diagonal()

In [ ]:
from petsc4py import PETSc
from slepc4py import SLEPc

A = Hr
k = 100
tol = 1e-2
max_iter = 100

### Setup matrix operator
n = A.shape[0]
mat = MatrixOperator(A)
A_operator = PETSc.Mat().createPython([n, n], mat)
A_operator.setUp()

### Solve eigenproblem
E = SLEPc.EPS()
E.create()
E.setOperators(A_operator)
E.setProblemType(SLEPc.EPS.ProblemType.HEP)
#E.setType(SLEPc.EPS.Type.LANCZOS)
E.setDimensions(k)
E.setTolerances(tol, max_iter)
E.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)

def monitor_fun(eps, iters, nconv, eigs, errors):
    print("Current iteration: %d, number of converged eigenvalues: %d" % (iters, nconv))

E.setMonitor(monitor_fun)

start = time.time()
E.solve()
print("Time_elapsed: %d" % (time.time() - start))
print("Number of calls to Ax: %d" % mat.n_calls)

Current iteration: 1, number of converged eigenvalues: 32
Current iteration: 2, number of converged eigenvalues: 39
Current iteration: 3, number of converged eigenvalues: 46
Current iteration: 4, number of converged eigenvalues: 56


In [8]:
### Collect results
print("")
its = E.getIterationNumber()
print("Number of iterations of the method: %i" % its)
sol_type = E.getType()
print("Solution method: %s" % sol_type)
nev, ncv, mpd = E.getDimensions()
print(nev, ncv, mpd)
print("Number of requested eigenvalues: %i" % nev)
tol, maxit = E.getTolerances()
print("Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit))
nconv = E.getConverged()
print("Number of converged eigenpairs: %d" % nconv)
nconv = min(nconv, k)

if nconv < k:
    raise ZeroDivisionError("Failed to converge for requested number of k with maxiter=%d" % max_iter)

vecs = np.zeros([n, nconv])
vals = np.zeros(nconv)

xr, tmp = A_operator.getVecs()
xi, tmp = A_operator.getVecs()

if nconv > 0:
    for i in range(nconv):
        val = E.getEigenpair(i, xr, xi)
        vals[i] = val.real
        vecs[:, i] = xr


Number of iterations of the method: 1
Solution method: lanczos
10 30 30
Number of requested eigenvalues: 10
Stopping condition: tol=0.01, maxit=100
Number of converged eigenpairs: 11


In [9]:
vals

array([-52.81019524, -11.47311446,  40.03498392,  44.04483584,
        47.52611907,  61.85944197,  66.21374104,  78.4068736 ,
        91.39344258, 112.57228353])

In [18]:
output_path = "../data/wiki/win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100"
np.save(output_path + ".vecs", vecs[:,:100])
np.save(output_path + ".vals", vals[:100])
np.save(output_path + ".degrees", degrees)

In [19]:
import shutil
base_path = '../data/wiki/'
shutil.copyfile(base_path + 'win=1_weighted_bethe_hessian_slepc_scaled_abs_tol=1e-3_pow=0.3_dim=500.words.vocab', 
                base_path + 'win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100' + ".words.vocab")

'../data/wiki/win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100.words.vocab'